In [ ]:
# https://claude.ai/chat/f134390c-d178-4865-9bbd-d8ceaa286447

## Install packages

Please install the following packages using uv or pip:

- pyserial
- google-genai

## Setup

- Connect the Arduino to your laptop using the USB cable.
- In the Arduno IDE, take note of the port that your Arduino is using.
- No other circuit wiring is needed because the LED that we will be using is part of the Arduino.

## Arduino code

This is the code that should be uploaded to the Arduino.

In [ ]:
# filename: switch-led-on-and-off.ino

"""


// Define the built-in LED pin for clarity
const int ledPin = LED_BUILTIN; 


void setup() {

  Serial.begin(9600);
  
  // Initialize the LED pin as an output
  pinMode(ledPin, OUTPUT);

  // Ensure LED is off at the start
  digitalWrite(ledPin, LOW);

}

void loop() {

   // ALWAYS-ON SERIAL COMMAND LISTENER 
  if (Serial.available() > 0) {
    
    String command = Serial.readStringUntil('\n');
    command.trim();
  
    // Handle the command to turn the LED ON
    if (command.equals("LED_ON")) {
      digitalWrite(ledPin, HIGH);
      Serial.println("OK: LED turned on"); // Send confirmation
    }
    // Handle the command to turn the LED OFF
    else if (command.equals("LED_OFF")) {
      digitalWrite(ledPin, LOW);
      Serial.println("OK: LED turned off"); // Send confirmation
    }
      
  }

}

"""


## --- Example 1: Switch LED ON and OFF ---

In this example we will create two Python functions. We will use them to switch the Arduino builtin LED on and off directly from this Jupyter notebook.

### Python code

In [1]:
# Python functions

import serial
import json
import time


# This needs to be changed to
# match the port you are using to
# connect to the Arduino.
PORT = '/dev/cu.usbserial-110'

BAUD_RATE = 9600


def switch_led_on(port=PORT, baud_rate=BAUD_RATE):

    # Construct the command based on the desired state
    command = f"LED_ON\n"

    # Establish serial connection
    ser = serial.Serial(port, baud_rate, timeout=2)
    time.sleep(2) # Wait for the connection to establish

    # Send the command to the Arduino
    ser.write(command.encode('utf-8'))
    print(f"Sent command: '{command.strip()}' to {port}")

    # Wait for and read the confirmation response
    response = ser.readline().decode('utf-8').strip()

    # Close the connection
    ser.close()
    
    if response:
        return f"Received response: '{response}'"
    else:
        return "No response"


def switch_led_off(port=PORT, baud_rate=BAUD_RATE):

    # Construct the command based on the desired state
    command = f"LED_OFF\n"

    # Establish serial connection
    ser = serial.Serial(port, baud_rate, timeout=2)
    time.sleep(2) # Wait for the connection to establish

    # Send the command to the Arduino
    ser.write(command.encode('utf-8'))
    print(f"Sent command: '{command.strip()}' to {port}")

    # Wait for and read the confirmation response
    response = ser.readline().decode('utf-8').strip()

    # Close the connection
    ser.close()
    
    if response:
        return f"Received response: '{response}'"
    else:
        return "No response"


### Switch the LED ON

In [2]:
# Switch the LED on
# (Run this cell to send a command to the Arduino)

arduino_response = switch_led_on(port=PORT, baud_rate=BAUD_RATE)

print(arduino_response)

Sent command: 'LED_ON' to /dev/cu.usbserial-110
Received response: 'OK: LED turned on'


### Switch the LED OFF

In [3]:
# Switch the LED off
# (Run this cell to send a command to the Arduino)

arduino_response = switch_led_off(port=PORT, baud_rate=BAUD_RATE)

print(arduino_response)

Sent command: 'LED_OFF' to /dev/cu.usbserial-110
Received response: 'OK: LED turned off'


## --- Example 2 - Integrate Arduino-Powered Tools into AI Chat ---

In this example we will send instructions to the Arduino while chatting with an Ai. This setup gives the Ai the ability to take actions in the physical world.

In [7]:
import os
import re

import serial
import time
from google import genai
from google.genai import types


In [8]:
# Add your API key here

GOOGLE_API_KEY = "YOUR_GOOGLE_API_KEY"

In [10]:
MODEL_ID = "gemini-2.5-flash"

PORT = '/dev/cu.usbserial-110'  # Change to match your Arduino port
BAUD_RATE = 9600


### Initialize the Genai Client and do a quick test

In [11]:
client = genai.Client(api_key=GOOGLE_API_KEY)

# Run a quick test

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Hello. How are you?"
)

print(response.text)

Hello! I'm functioning perfectly and ready to assist you.

How are you doing today?


## Set up functions and tool definitions

The functions are the same ones that we used in the previous example.

In [12]:

# -----------------------------
# Functions
# -----------------------------


def switch_led_on(port=PORT, baud_rate=BAUD_RATE):

    # Construct the command based on the desired state
    command = f"LED_ON\n"

    # Establish serial connection
    ser = serial.Serial(port, baud_rate, timeout=2)
    time.sleep(2) # Wait for the connection to establish

    # Send the command to the Arduino
    ser.write(command.encode('utf-8'))
    print(f"Sent command: '{command.strip()}' to {port}")

    # Wait for and read the confirmation response
    response = ser.readline().decode('utf-8').strip()

    # Close the connection
    ser.close()
    
    if response:
        return f"Received response: '{response}'"
    else:
        return "Error"


def switch_led_off(port=PORT, baud_rate=BAUD_RATE):

    # Construct the command based on the desired state
    command = f"LED_OFF\n"

    # Establish serial connection
    ser = serial.Serial(port, baud_rate, timeout=2)
    time.sleep(2) # Wait for the connection to establish

    # Send the command to the Arduino
    ser.write(command.encode('utf-8'))
    print(f"Sent command: '{command.strip()}' to {port}")

    # Wait for and read the confirmation response
    response = ser.readline().decode('utf-8').strip()

    # Close the connection
    ser.close()
    
    if response:
        return f"Received response: '{response}'"
    else:
        return "Error"


# -----------------------------
# Tool Declarations for Gemini
# -----------------------------

switch_led_on_declaration = {
    "name": "switch_led_on",
    "description": "Turns on the built-in LED on the Arduino board.",
    "parameters": {
        "type": "object",
        "properties": {}, 
        "required": []
    }
}


switch_led_off_declaration = {
    "name": "switch_led_off",
    "description": "Turns off the built-in LED on the Arduino board.",
    "parameters": {
        "type": "object",
        "properties": {}, 
        "required": []
    }
}



## Run a chat loop

Run the next cell to start the chat. Enter 'q' to quit.
Here we are using the Gemini chat API.

Suggestions:
- Please turn on the LED
- Please turn off the LED


### Note:

We are using the free Gemini version therefore this error happens regularly:

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

To keep the code simple I haven't included any error handling. When you encounter this error, please run the chat cell again and resend your message.

In [16]:

# Initialize the client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Set up tools
tools = types.Tool(
    function_declarations=[
        switch_led_on_declaration,
        switch_led_off_declaration,
    ]
)

config = types.GenerateContentConfig(
    system_instruction="You are a helpful assistant.",
    tools=[tools],
)


# Create a chat session
chat = client.chats.create(model=MODEL_ID, 
                           config=config)


while True:

    #print("==========")
    print()
    print("---USER---")
    user_input = input("User: ")
    #print("==========")

    if user_input.lower() == 'q':
        print("Goodbye!")
        break  # Exit the loop


    response = chat.send_message(user_input)

    # Check for function call
    if response.candidates[0].content.parts[0].function_call:
        
        function_call = response.candidates[0].content.parts[0].function_call

        print()
        print(f"Calling: {function_call.name}")
        
        # Execute function
        if function_call.name == "switch_led_on":
            result = switch_led_on()
            
        elif function_call.name == "switch_led_off":
            result = switch_led_off()
             

        print(f"Function_call_result: {result}")
        
        # Send result back - history is automatically maintained!
        function_response = types.Part.from_function_response(
            name=function_call.name,
            response={"result": result}
        )
        
        final_response = chat.send_message(function_response)


        print()
        print("---ASSISTANT---")
        print(final_response.text)
        
    else:

        print()
        print("---ASSISTANT---")
        print(response.text)


    


---USER---


User:  Hi



---ASSISTANT---
Hello! How can I help you today?


---USER---


User:  Please switch on the light



Calling: switch_led_on
Sent command: 'LED_ON' to /dev/cu.usbserial-110
Function_call_result: Received response: 'OK: LED turned on'

---ASSISTANT---
The light has been switched on.

---USER---


User:  q


Goodbye!
